In [ ]:
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.0 MB/s eta 0:00:00


In [28]:
import numpy as np
import math
from typing import Tuple
from collections import deque
import gymnasium as gym
from gymnasium import spaces
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback

In [29]:
# --- Environment Parameters ---
NUM_VEHICLES_TRAIN = 30
NUM_EDGE_SERVERS = 8
MAX_TASKS_PER_VEHICLE = 10
CPU_CYCLES_TASK_MCYCLES = (2, 20)
CPU_CYCLES_TASK_RANGE_MCY = (2, 20)
DATA_SIZE_TASK_Mbits = (2, 20)
DATA_SIZE_TASK_RANGE_MB = (2, 20)
VEHICLE_COMP_POWER_MCYCLES_PER_SEC = 1
EDGE_COMP_POWER_MCYCLES_PER_SEC = 2
CLOUD_COMP_POWER_MCYCLES_PER_SEC = 10

# Communication Parameters
VEHICLE_BANDWIDTH_MHZ = 100
EDGE_BANDWIDTH_MHZ = 100
CLOUD_BANDWIDTH_MHZ = 1000
VEHICLE_TRANSMIT_POWER_DBM = 1
VEHICLE_TX_POWER_DBM = 1
VEHICLE_EXECUTION_POWER_DBM = 3
EDGE_TX_POWER_DBM = 20
CLOUD_TX_POWER_DBM = 30
NOISE_POWER_DBM_PER_HZ = -174
POWER_CONSUMPTION_COEFFICIENT_XI = 1e-27
POWER_CONSUMPTION_COEFFICIENT_GAMMA = 2
RSSI_DBM = -75
TX_ANTENNA_GAIN_DBI = 20 # Transmit antenna gain
RX_ANTENNA_GAIN_DBI = -8 # Receive antenna gain
SIGNAL_ATTENUATION_DB = 7 # Signal attenuation caused by obstacles
WORKING_FREQUENCY_MHZ = 5000
SPEED_VEHICLES_MPS = 25  # m/s
MAX_VEHICLES = 50
RANDOM_SEED = 0
VEHICLE_CPU_FREQ_MHZ = 1
EDGE_CPU_FREQ_MHZ = 2
CLOUD_CPU_FREQ_MHZ = 10

# Mobility Model
HIGHWAY_LENGTH_M = 10000
VEHICLE_COVERAGE_RANGE_M = 500
EDGE_SERVER_COVERAGE_RANGE_M = 1000
EDGE_SERVER_LOCATIONS = [(i * HIGHWAY_LENGTH_M / (NUM_EDGE_SERVERS + 1), 0) for i in range(1, NUM_EDGE_SERVERS + 1)]

# Problem Formulation Weights
WEIGHT_COMPLETION_TIME = 0.5
WEIGHT_ENERGY_CONSUMPTION = 0.5
WEIGHT_DELAY = 0.5
DT = 1

# Constraints
MAX_TASK_DEADLINE_S = 2 # s
MAX_ENERGY_BUDGET_J = 10000 # Joules

# --- PPO Agent Parameters ---
LEARNING_RATE = 0.0003
N_STEPS = 2048
BATCH_SIZE = 64
N_EPOCHS = 10 # Number of epoch when optimizing the surrogate loss
GAMMA = 0.99 # Discount factor
GAE_LAMBDA = 0.95 # Factor for trade-off of bias vs variance for Generalized Advantage Estimator
CLIP_RANGE = 0.2 # Clipping parameter, for PPO
ENT_COEF = 0.01 # Entropy coefficient for the loss calculation
VF_COEF = 0.5 # Value function coefficient for the loss calculation

TOTAL_TIMESTEPS = 20000 # Total number of samples to train the agent

# --- Training Parameters ---
LOG_INTERVAL = 10 # Log every N episodes
SAVE_PATH = "results/trained_model_ppo.zip"
LOG_DIR = "results/"
LOG_FILE = "logs.csv"
os.makedirs(LOG_DIR, exist_ok=True)

# --- Testing Parameters ---
NUM_TEST_EPISODES = 100
TEST_VEHICLE_COUNTS = [10, 20, 30, 40, 50]
TEST_METHODS = ["MEPPO", "DDPG", "SAC", "Local-only", "Offloading-only", "Random"] # Add "PPO-no-priority", "PPO-no-dynamic-power" for ablation studies if needed

# --- Plotting Parameters ---
PLOT_TITLE_FONTSIZE = 14
PLOT_LABEL_FONTSIZE = 12
PLOT_LEGEND_FONTSIZE = 10

In [30]:
def dbm_to_mw(dbm: float) -> float:
    return 10.0 ** (dbm / 10.0)

def mw_to_dbm(mw: float) -> float:
    return 10.0 * math.log10(mw)

def dbm_to_watt(dbm: float) -> float:
    return dbm_to_mw(dbm) * 1e-3

def watt_to_dbm(watt: float) -> float:
    mw = watt * 1e3
    return mw_to_dbm(mw)


def calculate_snr_linear(received_power_watt: float, bandwidth_mhz: float,
                         interference_watt: float = 0.0,
                         noise_dbm_per_hz: float = NOISE_POWER_DBM_PER_HZ) -> float:

    bandwidth_hz = bandwidth_mhz * 1e6
    noise_w_per_hz = 10 ** ((noise_dbm_per_hz - 30.0) / 10.0)
    noise_total_w = noise_w_per_hz * bandwidth_hz
    denom = interference_watt + noise_total_w
    if denom <= 0:
        return 0.0
    return received_power_watt / denom

def calculate_data_rate_bps(bandwidth_mhz: float, snr_linear: float) -> float:
    if snr_linear <= 0:
        return 0.0
    bandwidth_hz = bandwidth_mhz * 1e6
    return bandwidth_hz * math.log2(1.0 + snr_linear)


def estimate_link_rate_and_snr(tx_power_dbm: float, distance_m: float, bandwidth_mhz: float,
                              tx_gain_dbi: float = None, rx_gain_dbi: float = None,
                              signal_attenuation_db: float = None, interference_watt: float = 0.0) -> Tuple[float, float]:

    recv_watt = dbm_to_watt(RSSI_DBM)
    snr_lin = calculate_snr_linear(recv_watt, bandwidth_mhz, interference_watt=interference_watt)
    rate_bps = calculate_data_rate_bps(bandwidth_mhz, snr_lin)
    return rate_bps, snr_lin

In [31]:
class Task:
    def __init__(self, task_id: int, cpu_cycles_mcycles: float, data_size_mb: float,
                 deadline_s: float = MAX_TASK_DEADLINE_S, origin_id: int = None):
        self.task_id = task_id
        self.origin_id = origin_id

        self.cpu_cycles_mcycles = float(cpu_cycles_mcycles)
        self.data_size_mb = float(data_size_mb)

        self.deadline_s = float(deadline_s)
        self.priority = 1
        self.offloading_decision = None   # 0=edge,1=vehicle,2=local,3=cloud
        self.assigned_entity = None

        self.remaining_cpu_mcycles = float(cpu_cycles_mcycles)
        self.remaining_data_mb = float(data_size_mb)

        self.start_time_s = None
        self.finish_time_s = None

        self.transmission_energy_j = 0.0
        self.computation_energy_j = 0.0

        self.done = False

    def __repr__(self):
        return (f"Task(id={self.task_id}, origin={self.origin_id}, cpu_Mc={self.cpu_cycles_mcycles:.2f}, "
                f"data_MB={self.data_size_mb:.2f}, rem_cpu={self.remaining_cpu_mcycles:.2f}, "
                f"rem_data={self.remaining_data_mb:.2f}, prio={self.priority}, done={self.done})")

    def mark_started(self, now_s):
        if self.start_time_s is None:
            self.start_time_s = now_s

    def mark_finished(self, now_s):
        self.finish_time_s = now_s
        self.done = True

In [32]:
class BaseComputeEntity:
    def __init__(self, entity_id, location=(0.0, 0.0), cpu_freq_mhz: float = 1.0):
        self.entity_id = entity_id
        self.location = np.array(location, dtype=float)
        self.cpu_freq_mhz = float(cpu_freq_mhz)

        self.task_queue = deque()
        self.current_transmissions = {}
        self.energy_consumed_j = 0.0
        self.last_update_time_s = 0.0
        self.processed_tasks_history = []

    def add_task_to_queue(self, task: Task, priority: int = 1):
        task.priority = priority
        self.task_queue.append((task, priority))
        self.task_queue = deque(sorted(list(self.task_queue), key=lambda x: x[1], reverse=True))

    def reset(self):
        self.task_queue.clear()
        self.current_transmissions.clear()
        self.energy_consumed_j = 0.0
        self.processed_tasks_history.clear()

    def get_current_workload_mcycles(self):
        return sum(t.remaining_cpu_mcycles for t, _ in self.task_queue)

    def process_step(self, dt_s: float, now_s: float):
        processed_tasks = []
        remaining_time = dt_s
        self.last_update_time_s = now_s

        while self.task_queue and remaining_time > 1e-12:
            task, priority = self.task_queue[0]
            cycles_can_do = self.cpu_freq_mhz * remaining_time

            if task.remaining_cpu_mcycles <= cycles_can_do:
                time_used = task.remaining_cpu_mcycles / (self.cpu_freq_mhz if self.cpu_freq_mhz>0 else 1.0)
                f_hz = self.cpu_freq_mhz * 1e6
                cycles_done = task.remaining_cpu_mcycles * 1e6
                energy_j = POWER_CONSUMPTION_COEFFICIENT_XI * (self.cpu_freq_mhz ** POWER_CONSUMPTION_COEFFICIENT_GAMMA) * task.remaining_cpu_mcycles
                energy_j *= 1000.0
                task.computation_energy_j += energy_j
                self.energy_consumed_j += energy_j

                task.remaining_cpu_mcycles = 0.0
                task.mark_finished(now_s + time_used)
                processed_tasks.append(task)
                self.task_queue.popleft()
                remaining_time -= time_used
            else:
                f_hz = self.cpu_freq_mhz * 1e6
                cycles_done = cycles_can_do * 1e6
                energy_j = POWER_CONSUMPTION_COEFFICIENT_XI * (self.cpu_freq_mhz ** POWER_CONSUMPTION_COEFFICIENT_GAMMA) * task.remaining_cpu_mcycles
                energy_j *= 1000.0
                task.remaining_cpu_mcycles -= cycles_can_do
                task.computation_energy_j += energy_j
                self.energy_consumed_j += energy_j
                remaining_time = 0.0

        return processed_tasks

In [33]:
class Vehicle(BaseComputeEntity):
    def __init__(self, entity_id, location=(0.0,0.0), cpu_freq_mhz: float = VEHICLE_COMP_POWER_MCYCLES_PER_SEC,
                 bandwidth_mhz: float = VEHICLE_BANDWIDTH_MHZ, tx_power_dbm: float = VEHICLE_TRANSMIT_POWER_DBM,
                 speed_mps: float = SPEED_VEHICLES_MPS):
        super().__init__(entity_id, location, cpu_freq_mhz)
        self.bandwidth_mhz = float(bandwidth_mhz)
        self.tx_power_dbm = float(tx_power_dbm)
        self.tx_power_watt = dbm_to_watt(self.tx_power_dbm)
        self.speed_mps = float(speed_mps)
        self.outgoing_transmissions = {}

    def __repr__(self):
        return f"Vehicle(id={self.entity_id}, loc={self.location.tolist()}, cpu={self.cpu_freq_mhz}MHz)"

    def move(self, dt: float):
        self.location[0] = (self.location[0] + self.speed_mps * dt) % HIGHWAY_LENGTH_M

    def start_transmission(self, task: Task, dest_entity: BaseComputeEntity, allocated_bw_mhz: float = None, tx_power_dbm: float = None):
        if allocated_bw_mhz is None:
            allocated_bw_mhz = self.bandwidth_mhz
        if tx_power_dbm is None:
            tx_power_dbm = self.tx_power_dbm

        tx_info = {
            'task': task,
            'dest': dest_entity,
            'allocated_bw_mhz': float(allocated_bw_mhz),
            'tx_power_dbm': float(tx_power_dbm),
            'progress_mb': 0.0,
            'started': True
        }
        self.outgoing_transmissions[task.task_id] = tx_info
        task.offloading_decision = 1 if isinstance(dest_entity, Vehicle) else (0 if dest_entity.__class__.__name__ == 'EdgeServer' else 3)
        task.assigned_entity = dest_entity.entity_id

    def progress_transmissions(self, dt: float, interference_watt: float = 0.0):
        completed_transfers = []
        to_remove = []
        for tid, info in list(self.outgoing_transmissions.items()):
            task = info['task']
            dest = info['dest']
            bw = info['allocated_bw_mhz']
            tx_dbm = info['tx_power_dbm']

            distance_m = np.linalg.norm(self.location - dest.location)
            rate_bps, snr_lin = estimate_link_rate_and_snr(tx_dbm, distance_m, bw)
            rate_mbps = rate_bps / 1e6
            data_sent_mb = (rate_mbps * dt) / 8.0

            info['progress_mb'] += data_sent_mb
            task.remaining_data_mb = max(0.0, task.remaining_data_mb - data_sent_mb)

            tx_power_watt = dbm_to_watt(tx_dbm)
            energy_j = tx_power_watt * dt
            energy_j *= 1000.0
            self.energy_consumed_j += energy_j
            task.transmission_energy_j += energy_j

            if task.remaining_data_mb <= 1e-12:
                completed_transfers.append((task, dest))
                to_remove.append(tid)

        for tid in to_remove:
            del self.outgoing_transmissions[tid]

        for task, dest in completed_transfers:
            dest.add_task_to_queue(task, priority=task.priority)

        return completed_transfers

In [34]:
class EdgeServer(BaseComputeEntity):
    def __init__(self, entity_id, location=(0.0, 0.0), cpu_freq_mhz: float = EDGE_COMP_POWER_MCYCLES_PER_SEC,
                 bandwidth_mhz: float = EDGE_BANDWIDTH_MHZ, tx_power_dbm: float = EDGE_TX_POWER_DBM):
        super().__init__(entity_id, location, cpu_freq_mhz)
        self.bandwidth_mhz = float(bandwidth_mhz)
        self.tx_power_dbm = float(tx_power_dbm)
        self.tx_power_watt = dbm_to_watt(self.tx_power_dbm)
        self.connected_vehicles = set()

    def __repr__(self):
        return f"EdgeServer(id={self.entity_id}, loc={self.location.tolist()}, cpu={self.cpu_freq_mhz}MHz)"


class CloudServer(BaseComputeEntity):
    def __init__(self, entity_id="cloud_0", location=(HIGHWAY_LENGTH_M/2, 5000.0), cpu_freq_mhz: float = CLOUD_COMP_POWER_MCYCLES_PER_SEC,
                 bandwidth_mhz: float = CLOUD_BANDWIDTH_MHZ, tx_power_dbm: float = CLOUD_TX_POWER_DBM):
        super().__init__(entity_id, location, cpu_freq_mhz)
        self.bandwidth_mhz = float(bandwidth_mhz)
        self.tx_power_dbm = float(tx_power_dbm)
        self.tx_power_watt = dbm_to_watt(self.tx_power_dbm)

    def __repr__(self):
        return f"CloudServer(id={self.entity_id}, loc={self.location.tolist()}, cpu={self.cpu_freq_mhz}MHz)"

In [38]:
class VECEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_vehicles: int = 10, max_tasks_per_vehicle: int = MAX_TASKS_PER_VEHICLE, episode_duration_s: int = 200):
        super().__init__()
        self.num_vehicles = min(num_vehicles, MAX_VEHICLES)  # محدود به max
        self.max_tasks_per_vehicle = max_tasks_per_vehicle
        self.episode_duration_s = episode_duration_s
        self.current_time_s = 0.0
        self.vehicles = {}
        self.edges = {}
        self.cloud = None
        self.all_tasks = {}
        self.task_counter = 0
        self._init_entities()
        obs_dim = 3 * MAX_VEHICLES + 3  # ثابت برای max
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(obs_dim,), dtype=np.float32)
        self.action_space = spaces.MultiDiscrete([4] * MAX_VEHICLES)  # ثابت برای max
        self.seed(RANDOM_SEED)

    def _init_entities(self):
        for i, loc in enumerate(EDGE_SERVER_LOCATIONS):
            es = EdgeServer(entity_id=f"es_{i}", location=loc, cpu_freq_mhz=EDGE_CPU_FREQ_MHZ, bandwidth_mhz=EDGE_BANDWIDTH_MHZ, tx_power_dbm=EDGE_TX_POWER_DBM)
            self.edges[i] = es
        self.cloud = CloudServer(entity_id="cloud_0", location=(HIGHWAY_LENGTH_M/2, 5000.0), cpu_freq_mhz=CLOUD_CPU_FREQ_MHZ, bandwidth_mhz=CLOUD_BANDWIDTH_MHZ, tx_power_dbm=CLOUD_TX_POWER_DBM)
        for i in range(self.num_vehicles):
            x = float(np.random.uniform(0, HIGHWAY_LENGTH_M))
            v = Vehicle(entity_id=f"v_{i}", location=(x, 0.0), cpu_freq_mhz=VEHICLE_CPU_FREQ_MHZ, bandwidth_mhz=VEHICLE_BANDWIDTH_MHZ, tx_power_dbm=VEHICLE_TX_POWER_DBM, speed_mps=SPEED_VEHICLES_MPS)
            self.vehicles[i] = v

    def seed(self, s=None):
        np.random.seed(s)
        random.seed(s)

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)
        self.current_time_s = 0.0
        self.all_tasks.clear()
        self.task_counter = 0
        for v in self.vehicles.values():
            v.reset()
        for es in self.edges.values():
            es.reset()
        self.cloud.reset()
        for i, v in self.vehicles.items():
            num = np.random.randint(0, self.max_tasks_per_vehicle + 1)
            for _ in range(num):
                cpu = float(np.random.uniform(*CPU_CYCLES_TASK_RANGE_MCY))
                data = float(np.random.uniform(*DATA_SIZE_TASK_RANGE_MB))
                t = Task(task_id=self.task_counter, cpu_cycles_mcycles=cpu, data_size_mb=data, origin_id=v.entity_id)
                self.task_counter += 1
                v.add_task_to_queue(t, priority=1)
                self.all_tasks[t.task_id] = t
        obs = self._get_observation()
        return obs, {}

    def _get_observation(self):
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        for i in range(MAX_VEHICLES):
            v = self.vehicles.get(i)
            idx = 3 * i
            if v is not None:
                obs[idx] = float(v.location[0] / HIGHWAY_LENGTH_M)
                obs[idx+1] = float(v.get_current_workload_mcycles() / (self.max_tasks_per_vehicle * CPU_CYCLES_TASK_RANGE_MCY[1] + 1e-9))
                obs[idx+2] = float(len(v.task_queue) / (self.max_tasks_per_vehicle + 1))
            # else: pad with 0 (already is)
        # global features (unchanged)
        avg_edge = np.mean([es.get_current_workload_mcycles() for es in self.edges.values()]) if self.edges else 0.0
        obs[-3] = float(avg_edge / (self.max_tasks_per_vehicle * CPU_CYCLES_TASK_RANGE_MCY[1] + 1e-9))
        obs[-2] = float(self.cloud.get_current_workload_mcycles() / (self.max_tasks_per_vehicle * CPU_CYCLES_TASK_RANGE_MCY[1] + 1e-9))
        avg_vehicle = np.mean([v.get_current_workload_mcycles() for v in self.vehicles.values()]) if self.vehicles else 0.0
        obs[-1] = float(avg_vehicle / (self.max_tasks_per_vehicle * CPU_CYCLES_TASK_RANGE_MCY[1] + 1e-9))
        return obs

    def step(self, action):
        action = action[:self.num_vehicles]  # Trim to current num_vehicles
        assert len(action) == self.num_vehicles
        for i, act in enumerate(action):
            v = self.vehicles.get(i)
            if not v or not v.task_queue:
                continue
            task_tuple = v.task_queue[0]
            if isinstance(task_tuple, tuple):
                task, pr = task_tuple
            else:
                task = task_tuple
                pr = 1
            task.mark_started(self.current_time_s)  # شروع زمان
            if act == 2:  # local
                task.offloading_decision = 2
                task.assigned_entity = v.entity_id
            elif act == 0:  # edge
                dists = [(eid, np.linalg.norm(v.location - es.location)) for eid, es in self.edges.items()]
                if dists:
                    eid, mind = min(dists, key=lambda x: x[1])
                    es = self.edges[eid]
                    v.start_transmission(task, es, allocated_bw_mhz=v.bandwidth_mhz, tx_power_dbm=v.tx_power_dbm)
            elif act == 1:  # other vehicle
                candidates = [(j, np.linalg.norm(v.location - v2.location)) for j, v2 in self.vehicles.items() if j != i]
                if candidates:
                    j, dmin = min(candidates, key=lambda x: x[1])
                    v2 = self.vehicles[j]
                    v.start_transmission(task, v2, allocated_bw_mhz=min(v.bandwidth_mhz, v2.bandwidth_mhz), tx_power_dbm=v.tx_power_dbm)
            elif act == 3:  # cloud
                 v.start_transmission(task, self.cloud, allocated_bw_mhz=min(v.bandwidth_mhz, self.cloud.bandwidth_mhz), tx_power_dbm=v.tx_power_dbm)
        self._simulate_one_step(DT)
        reward = self._compute_reward()

        completed_tasks = [t for t in self.all_tasks.values() if t.done and t.start_time_s is not None and t.finish_time_s is not None]
        avg_completion_time = np.mean([t.finish_time_s - t.start_time_s for t in completed_tasks]) if completed_tasks else 0.0
        total_energy = sum(v.energy_consumed_j for v in self.vehicles.values()) + \
                       sum(es.energy_consumed_j for es in self.edges.values()) + \
                       self.cloud.energy_consumed_j
        self.current_time_s += DT
        obs = self._get_observation()
        terminated = (self.current_time_s >= self.episode_duration_s)
        truncated = False
        info = {"time": self.current_time_s, "avg_completion_time": avg_completion_time, "total_energy": total_energy}
        return obs, reward, terminated, truncated, info

    def _simulate_one_step(self, dt):
        for v in self.vehicles.values():
            v.move(dt)
            v.progress_transmissions(dt)
            try:
                v.process_step(dt, self.current_time_s)
            except TypeError:
                v.process_step(dt)
        for es in self.edges.values():
            try:
                es.process_step(dt, self.current_time_s)
            except TypeError:
                es.process_step(dt)
        try:
            self.cloud.process_step(dt, self.current_time_s)
        except TypeError:
            self.cloud.process_step(dt)

    def _compute_reward(self):
        total_energy = sum(v.energy_consumed_j for v in self.vehicles.values()) + \
                       sum(es.energy_consumed_j for es in self.edges.values()) + \
                       self.cloud.energy_consumed_j
        total_remaining_deadline = sum((max(0.0, t.deadline_s - (self.current_time_s - (t.start_time_s if t.start_time_s else 0.0))) for t in self.all_tasks.values() if not t.done))
        n_tasks = max(1.0, float(len(self.all_tasks)))
        n_vehicles = max(1.0, float(len(self.vehicles)))
        r = - (WEIGHT_DELAY * (total_remaining_deadline / n_tasks) + WEIGHT_ENERGY * (total_energy / n_vehicles))
        return r

    def render(self, mode='human'):
        s = f"Time={self.current_time_s:.1f}s\n"
        for i, v in self.vehicles.items():
            s += f"V{i}: pos={v.location[0]:.1f}, q={len(v.task_queue)}, E={v.energy_consumed_j:.4f}\n"
        print(s)

    def close(self):
        pass

In [39]:
env = VECEnv(num_vehicles=5, max_tasks_per_vehicle=3, episode_duration_s=20)
obs, info = env.reset()
print("obs shape:", obs.shape)
for step in range(5):
    a = env.action_space.sample()
    obs, r, terminated, truncated, info = env.step(a)
    done = terminated or truncated
    print(f"step {step}, reward={r:.4f}, time={info['time']}")
    env.render()
    if done: break

obs shape: (153,)
step 0, reward=-1.3777, time=1.0
Time=1.0s
V0: pos=6530.0, q=0, E=0.0000
V1: pos=7731.7, q=3, E=1.2589
V2: pos=2260.7, q=2, E=0.0000
V3: pos=2974.6, q=2, E=1.2589
V4: pos=5673.9, q=3, E=1.2589

step 1, reward=-1.3312, time=2.0
Time=2.0s
V0: pos=6555.0, q=0, E=0.0000
V1: pos=7756.7, q=3, E=2.5179
V2: pos=2285.7, q=3, E=1.2589
V3: pos=2999.6, q=3, E=2.5179
V4: pos=5698.9, q=3, E=2.5179

step 2, reward=-1.2589, time=3.0
Time=3.0s
V0: pos=6580.0, q=2, E=0.0000
V1: pos=7781.7, q=2, E=3.7768
V2: pos=2310.7, q=3, E=1.2589
V3: pos=3024.6, q=3, E=3.7768
V4: pos=5723.9, q=3, E=3.7768

step 3, reward=-1.8625, time=4.0
Time=4.0s
V0: pos=6605.0, q=1, E=1.2589
V1: pos=7806.7, q=2, E=3.7768
V2: pos=2335.7, q=3, E=2.5179
V3: pos=3049.6, q=3, E=5.0357
V4: pos=5748.9, q=3, E=5.0357

step 4, reward=-2.4902, time=5.0
Time=5.0s
V0: pos=6630.0, q=1, E=2.5179
V1: pos=7831.7, q=2, E=3.7768
V2: pos=2360.7, q=4, E=2.5179
V3: pos=3074.6, q=3, E=6.2946
V4: pos=5773.9, q=3, E=6.2946



In [40]:
def train_ppo(total_timesteps=TOTAL_TIMESTEPS, num_vehicles=8, save_path=SAVE_PATH, log_dir=LOG_DIR):
    print("Initializing environment and PPO agent...")

    env = DummyVecEnv([lambda: Monitor(VECEnv(num_vehicles=num_vehicles))])

    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=LEARNING_RATE,
        n_steps=N_STEPS,
        batch_size=BATCH_SIZE,
        n_epochs=N_EPOCHS,
        gamma=GAMMA,
        gae_lambda=GAE_LAMBDA,
        clip_range=CLIP_RANGE,
        ent_coef=ENT_COEF,
        vf_coef=VF_COEF,
        verbose=1,
        tensorboard_log=log_dir
    )

    ckpt_dir = os.path.join(log_dir, "checkpoints")
    os.makedirs(ckpt_dir, exist_ok=True)
    checkpoint_callback = CheckpointCallback(save_freq=5000, save_path=ckpt_dir, name_prefix="ppo_veh")

    print(f"Training for {total_timesteps} timesteps...")
    model.learn(total_timesteps=total_timesteps, callback=checkpoint_callback)

    model.save(save_path)
    print(f" Model saved at: {save_path}")
    return model

model = train_ppo(total_timesteps=20000, num_vehicles=8)

Initializing environment and PPO agent...
Using cpu device
Training for 20000 timesteps...
Logging to results/PPO_3
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.95e+03 |
| time/              |           |
|    fps             | 77        |
|    iterations      | 1         |
|    time_elapsed    | 26        |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.92e+03   |
| time/                   |             |
|    fps                  | 66          |
|    iterations           | 2           |
|    time_elapsed         | 61          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014407932 |
|    clip_fraction        | 0.184       |
|    clip_range           | 0.

In [41]:
def evaluate_model(model_path, num_episodes=5, vehicle_ranges=[10, 20, 30, 40, 50], render=False):
    print(f"Loading model from {model_path}")
    model = PPO.load(model_path)

    results = {
        'num_vehicles': vehicle_ranges,
        'avg_completion_times': [],
        'avg_total_energies': [],
        'avg_rewards': []
    }

    for n in vehicle_ranges:
        env = VECEnv(num_vehicles=n, max_tasks_per_vehicle=MAX_TASKS_PER_VEHICLE, episode_duration_s=200)
        episode_completion_times = []
        episode_total_energies = []
        episode_rewards = []

        for ep in range(num_episodes):
            obs, _ = env.reset()
            terminated, truncated = False, False
            ep_reward = 0.0
            while not (terminated or truncated):
                action, _states = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                if render:
                    env.render()

            episode_completion_times.append(info.get('avg_completion_time', 0.0))
            episode_total_energies.append(info.get('total_energy', 0.0))
            episode_rewards.append(ep_reward)
            print(f"Episode {ep+1} (vehicles={n}): Completion Time={info['avg_completion_time']:.3f}s, Energy={info['total_energy']:.3f}J, Reward={ep_reward:.3f}")


        results['avg_completion_times'].append(np.mean(episode_completion_times))
        results['avg_total_energies'].append(np.mean(episode_total_energies))
        results['avg_rewards'].append(np.mean(episode_rewards))
        print(f"\nMean for {n} vehicles: Completion Time={results['avg_completion_times'][-1]:.3f}s, Energy={results['avg_total_energies'][-1]:.3f}J, Reward={results['avg_rewards'][-1]:.3f}")

    return results

results = evaluate_model("results/trained_model_ppo.zip", num_episodes=3)
import pickle
with open("results.pkl", "wb") as f:
    pickle.dump(results, f)


Loading model from results/trained_model_ppo.zip
Episode 1 (vehicles=10): Completion Time=9.778s, Energy=313.472J, Reward=-2662.246
Episode 2 (vehicles=10): Completion Time=7.882s, Energy=180.026J, Reward=-1623.681
Episode 3 (vehicles=10): Completion Time=6.344s, Energy=162.401J, Reward=-1512.151

Mean for 10 vehicles: Completion Time=8.002s, Energy=218.633J, Reward=-1932.693
Episode 1 (vehicles=20): Completion Time=7.643s, Energy=577.847J, Reward=-2608.277
Episode 2 (vehicles=20): Completion Time=6.734s, Energy=387.749J, Reward=-1748.929
Episode 3 (vehicles=20): Completion Time=9.685s, Energy=688.632J, Reward=-3013.307

Mean for 20 vehicles: Completion Time=8.021s, Energy=551.409J, Reward=-2456.838
Episode 1 (vehicles=30): Completion Time=8.067s, Energy=741.507J, Reward=-2249.388
Episode 2 (vehicles=30): Completion Time=7.722s, Energy=784.311J, Reward=-2307.211
Episode 3 (vehicles=30): Completion Time=10.189s, Energy=621.909J, Reward=-1810.605

Mean for 30 vehicles: Completion Time=8.

In [ ]:
import pickle
with open("results.pkl", "rb") as f:
    results = pickle.load(f)


In [42]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

def plot_results_safe(results):
    num_vehicles = results['num_vehicles']
    avg_completion_times = results['avg_completion_times']
    avg_total_energies = results['avg_total_energies']

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    axes[0].plot(num_vehicles, avg_completion_times, marker='o')
    axes[0].set_xlabel('Number of Vehicles')
    axes[0].set_ylabel('Average Completion Time (s)')
    axes[0].set_title('Completion Time vs Vehicles')
    axes[0].grid(True)

    axes[1].plot(num_vehicles, avg_total_energies, marker='o', color='orange')
    axes[1].set_xlabel('Number of Vehicles')
    axes[1].set_ylabel('Total Energy (J)')
    axes[1].set_title('Energy Consumption vs Vehicles')
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig("results_summary.png", dpi=150)
    plt.close(fig)

plot_results_safe(results)
print(" Chart saved as results_summary.png")


 Chart saved as results_summary.png


In [44]:
def evaluate_baselines(env, method, num_episodes=3):
    episode_completion_times = []
    episode_total_energies = []
    episode_rewards = []
    for ep in range(num_episodes):
        obs, _ = env.reset()
        terminated, truncated = False, False
        ep_reward = 0.0
        while not (terminated or truncated):
            if method == "Random":
                action = env.action_space.sample()[:env.num_vehicles]
            elif method == "Local-only":
                action = [2] * env.num_vehicles
            elif method == "Cloud-only":
                action = [3] * env.num_vehicles
            elif method == "Edge-only":
                action = [0] * env.num_vehicles
            obs, reward, terminated, truncated, info = env.step(action)
            ep_reward += reward
        episode_completion_times.append(info.get('avg_completion_time', 0.0))
        episode_total_energies.append(info.get('total_energy', 0.0))
        episode_rewards.append(ep_reward)
    return np.mean(episode_completion_times), np.mean(episode_total_energies), np.mean(episode_rewards)

def evaluate_all_methods(model_path, num_episodes=3, vehicle_ranges=[10, 20, 30, 40, 50], methods=["MEPPO", "Random", "Local-only", "Cloud-only"]):
    model = PPO.load(model_path) if "MEPPO" in methods else None
    all_results = {method: {'num_vehicles': vehicle_ranges, 'avg_completion_times': [], 'avg_total_energies': []} for method in methods}

    for n in vehicle_ranges:
        env = VECEnv(num_vehicles=n)
        for method in methods:
            if method == "MEPPO":
                avg_time, avg_energy, _ = evaluate_model_helper(model, env, num_episodes)  # helper for MEPPO
            else:
                avg_time, avg_energy, _ = evaluate_baselines(env, method, num_episodes)
            all_results[method]['avg_completion_times'].append(avg_time)
            all_results[method]['avg_total_energies'].append(avg_energy)
            print(f"{method} for {n} vehicles: Time={avg_time:.3f}s, Energy={avg_energy:.3f}J")

    return all_results

def evaluate_model_helper(model, env, num_episodes):
    episode_completion_times = []
    episode_total_energies = []
    episode_rewards = []
    for ep in range(num_episodes):
        obs, _ = env.reset()
        terminated, truncated = False, False
        ep_reward = 0.0
        while not (terminated or truncated):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            ep_reward += reward
        episode_completion_times.append(info.get('avg_completion_time', 0.0))
        episode_total_energies.append(info.get('total_energy', 0.0))
        episode_rewards.append(ep_reward)
    return np.mean(episode_completion_times), np.mean(episode_total_energies), np.mean(episode_rewards)


all_results = evaluate_all_methods("results/trained_model_ppo.zip")

MEPPO for 10 vehicles: Time=11.474s, Energy=230.383J
Random for 10 vehicles: Time=11.211s, Energy=236.678J
Local-only for 10 vehicles: Time=10.667s, Energy=0.000J
Cloud-only for 10 vehicles: Time=5.304s, Energy=255.982J
MEPPO for 20 vehicles: Time=11.422s, Energy=582.882J
Random for 20 vehicles: Time=11.257s, Energy=441.044J
Local-only for 20 vehicles: Time=10.879s, Energy=0.000J
Cloud-only for 20 vehicles: Time=9.102s, Energy=737.730J
MEPPO for 30 vehicles: Time=10.460s, Energy=739.829J
Random for 30 vehicles: Time=14.645s, Energy=817.462J
Local-only for 30 vehicles: Time=10.878s, Energy=0.000J
Cloud-only for 30 vehicles: Time=11.739s, Energy=1357.541J
MEPPO for 40 vehicles: Time=10.587s, Energy=938.739J
Random for 40 vehicles: Time=15.352s, Energy=1221.997J
Local-only for 40 vehicles: Time=10.536s, Energy=0.000J
Cloud-only for 40 vehicles: Time=12.809s, Energy=1827.120J
MEPPO for 50 vehicles: Time=10.716s, Energy=1276.970J
Random for 50 vehicles: Time=17.804s, Energy=1721.790J
Local-

In [45]:

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

def plot_results_comparison(all_results):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    colors = {'MEPPO': 'blue', 'Random': 'green', 'Local-only': 'red', 'Cloud-only': 'orange'}
    methods = list(all_results.keys())
    num_vehicles = all_results['MEPPO']['num_vehicles']
    bar_width = 2
    index = np.arange(len(num_vehicles))

    # Completion Time
    for i, method in enumerate(methods):
        offset = i * bar_width - (len(methods) - 1) * bar_width / 2
        axes[0].bar(index + offset, all_results[method]['avg_completion_times'], bar_width, label=method, color=colors.get(method))
    axes[0].set_xlabel('Number of Vehicles')
    axes[0].set_ylabel('Average Completion Time (s)')
    axes[0].set_title('Completion Time vs Vehicles')
    axes[0].set_xticks(index)
    axes[0].set_xticklabels(num_vehicles)
    axes[0].legend()
    axes[0].grid(True)

    # Energy Consumption
    for i, method in enumerate(methods):
        offset = i * bar_width - (len(methods) - 1) * bar_width / 2
        axes[1].bar(index + offset, all_results[method]['avg_total_energies'], bar_width, label=method, color=colors.get(method))
    axes[1].set_xlabel('Number of Vehicles')
    axes[1].set_ylabel('Total Energy (J)')
    axes[1].set_title('Energy Consumption vs Vehicles')
    axes[1].set_xticks(index)
    axes[1].set_xticklabels(num_vehicles)
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig("results_comparison_bar.png", dpi=150)
    plt.close(fig)
    print("✅ Comparison chart saved as results_comparison_bar.png")


plot_results_comparison(all_results)

✅ Comparison chart saved as results_comparison_bar.png


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
